# Recidivism Prediction

## Importing Libraries and Dataset

In [25]:
# Import Libraries
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sqlalchemy import create_engine
import pymysql
import psycopg2 as pg

In [16]:
# Read Cleaned Data from Segment_2_Michael branch
prisoner_df = pd.read_csv("https://raw.githubusercontent.com/MFairbro1/Recidivism_Prediction/Segment_2_Michael/Resources/Recidivism_Cleaned.csv")
prisoner_df = prisoner_df.iloc[: , 1:]

In [17]:
# Top 5 Rows of DataFrame
prisoner_df.head()

,Race_Asian,Race_Black,Race_Hispanic,Race_Native,Race_Other,Race_White,Age_25-34,Age_35-44,Age_45-54,Age_55 and Older,...,Release Type_Discharged End of Sentence,Release Type_Interstate Compact Parole,Release Type_Parole,Release Type_Paroled to Detainer - INS,Release Type_Paroled to Detainer - Iowa,Release Type_Paroled to Detainer - Out of State,Release Type_Paroled to Detainer - U.S. Marshall,Release Type_Special Sentence,Year Released,Recidivism
0,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,2010,1
1,0,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,2010,1
2,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,2010,1
3,0,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,0,0,0,2010,1
4,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,2010,1


## Exploring Dataset

In [18]:
# Data Types of All Variables in DataFrame
prisoner_df.dtypes

Race_Asian                                                         int64
Race_Black                                                         int64
Race_Hispanic                                                      int64
Race_Native                                                        int64
Race_Other                                                         int64
Race_White                                                         int64
Age_25-34                                                          int64
Age_35-44                                                          int64
Age_45-54                                                          int64
Age_55 and Older                                                   int64
Age_Under 25                                                       int64
Offense Classification_A Felony                                    int64
Offense Classification_Aggravated Misdemeanor                      int64
Offense Classification_B Felony                    

In [19]:
prisoner_df['Recidivism'].value_counts()

0    17339
1     8681
Name: Recidivism, dtype: int64

## Machine Learning

### Setting Target and Feature Variables

In [20]:
# Setting 'Recidivism' as the target variable for the machine learning algorithm
y = prisoner_df["Recidivism"].values.reshape(-1, 1)

In [21]:
# Features
X = prisoner_df.copy()
X = X.drop("Recidivism", axis = 1)
X.head()

,Race_Asian,Race_Black,Race_Hispanic,Race_Native,Race_Other,Race_White,Age_25-34,Age_35-44,Age_45-54,Age_55 and Older,...,Offense Type_Violent,Release Type_Discharged End of Sentence,Release Type_Interstate Compact Parole,Release Type_Parole,Release Type_Paroled to Detainer - INS,Release Type_Paroled to Detainer - Iowa,Release Type_Paroled to Detainer - Out of State,Release Type_Paroled to Detainer - U.S. Marshall,Release Type_Special Sentence,Year Released
0,0,0,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,2010
1,0,0,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,2010
2,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,2010
3,0,0,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,2010
4,0,1,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,2010


### Splitting Training and Testing Data

In [22]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=21, test_size=0.2)

In [23]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(20816, 40)
(5204, 40)
(20816, 1)
(5204, 1)


In [26]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [27]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [28]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)